In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#lyft_df = pd.read_csv('Data_files/lyft.csv')
#uber_df = pd.read_csv('Data_files/uber.csv')
uber_lyft_df = pd.read_csv('Data_files/rideshare_data.csv')

In [2]:
#Spencer's code 


In [3]:
#Adam's code

In [4]:
#Rory's code
uber_lyft_df.head()

,Unnamed: 0,id,hour,day,month,source,destination,cab_type,name,price,...,visibility,temperatureHigh,temperatureLow,dewPoint,windBearing,cloudCover,uvIndex,visibility.1,moonPhase,weekday
0,1,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,9,16,12,Haymarket Square,North Station,Lyft,Shared,5.0,...,10.000,43.68,34.19,32.70,57,0.72,0,10.000,0.30,Sun
1,2,4bd23055-6827-41c6-b23b-3c491f24e74d,2,27,11,Haymarket Square,North Station,Lyft,Lux,11.0,...,4.786,47.30,42.10,41.83,90,1.00,0,4.786,0.64,Tue
2,3,981a3613-77af-4620-a42a-0c0866077d1e,1,28,11,Haymarket Square,North Station,Lyft,Lyft,7.0,...,10.000,47.55,33.10,31.10,240,0.03,0,10.000,0.68,Wed
3,4,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,4,30,11,Haymarket Square,North Station,Lyft,Lux Black XL,26.0,...,10.000,45.03,28.90,26.64,310,0.00,0,10.000,0.75,Fri
4,5,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,3,29,11,Haymarket Square,North Station,Lyft,Lyft XL,9.0,...,10.000,42.18,36.71,28.61,303,0.44,0,10.000,0.72,Thu


In [5]:
#lyft_df.head()

In [6]:
# Final DataFrame
final_uber_lyft_df = uber_lyft_df[["id", "cab_type", "name", "price", "distance"]]
final_uber_lyft_df.head()

,id,cab_type,name,price,distance
0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,Lyft,Shared,5.0,0.44
1,4bd23055-6827-41c6-b23b-3c491f24e74d,Lyft,Lux,11.0,0.44
2,981a3613-77af-4620-a42a-0c0866077d1e,Lyft,Lyft,7.0,0.44
3,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,Lyft,Lux Black XL,26.0,0.44
4,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,Lyft,Lyft XL,9.0,0.44


In [7]:
# Sort DataFrame
#final_uber_lyft_df.sort_values(['cab_type','name'], ascending=True)
final_uber_lyft_df.sort_values(['cab_type','name'], ascending=[1,0])

,id,cab_type,name,price,distance
0,424553bb-7174-41ea-aeb4-fe06d4f4b9d7,Lyft,Shared,5.0,0.44
8,4f9fee41-fde3-4767-bbf1-a00e108701fb,Lyft,Shared,3.0,1.08
22,a7c1afce-9781-41c9-b6a6-df109ad43ccd,Lyft,Shared,3.5,0.72
32,00dd58fc-fa9e-4eeb-bb6e-dfeb94611e85,Lyft,Shared,3.5,3.24
44,0c26c153-a58d-419d-b557-37cfad67585c,Lyft,Shared,5.0,1.76
...,...,...,...,...,...
693006,39544c00-20c2-4a8c-9248-5818d37c4af9,Uber,Black,36.0,7.36
693027,8fda0334-9e8a-40bb-9251-76eede9be5dd,Uber,Black,22.0,2.50
693046,73510473-814f-42a0-9641-3db6b01bf059,Uber,Black,16.0,0.91
693057,0f13e495-cd0a-4b87-8219-b9a4ae06ece7,Uber,Black,17.0,1.61


In [8]:
#final_uber_lyft_df.describe

In [9]:
#final_lyft_df = lyft_df[["id", "cab_type", "name", "price", "distance"]]
#final_lyft_df.head()
#final_uber_lyft_df.loc[final_uber_lyft_df['cab_type'] == "Uber"]
#final_uber_lyft_df.loc[((final_uber_lyft_df['cab_type'] == "Lyft") | (final_uber_lyft_df['cab_type'] == "Uber"))
#                      & ((final_uber_lyft_df['name'] == "Lux") | (final_uber_lyft_df['name'] == "Lux Black XL")
#                      |  (final_uber_lyft_df['name'] == "Lyft XL") | (final_uber_lyft_df['name'] == "UberX")
#                      |  (final_uber_lyft_df['name'] == "Black SUV") | (final_uber_lyft_df['name'] == "UberXL"))]
final_uber_lyft_df.loc[((final_uber_lyft_df['cab_type'] == "Lyft") | (final_uber_lyft_df['cab_type'] == "Uber"))
                      & ((final_uber_lyft_df['name'] != "Shared") & (final_uber_lyft_df['name'] != "Taxi")
                      &  (final_uber_lyft_df['name'] != "Lyft") & (final_uber_lyft_df['name'] != "Uber Pool")
                      &  (final_uber_lyft_df['name'] != "WAV"))]

,id,cab_type,name,price,distance
1,4bd23055-6827-41c6-b23b-3c491f24e74d,Lyft,Lux,11.0,0.44
3,c2d88af2-d278-4bfd-a8d0-29ca77cc5512,Lyft,Lux Black XL,26.0,0.44
4,e0126e1f-8ca9-4f2e-82b3-50505a09db9a,Lyft,Lyft XL,9.0,0.44
5,f6f6d7e4-3e18-4922-a5f5-181cdd3fa6f2,Lyft,Lux Black,16.5,0.44
6,462816a3-820d-408b-8549-0b39e82f65ac,Lyft,Lyft XL,10.5,1.08
...,...,...,...,...,...
693064,25ed29df-4f9c-4aac-8218-681427f1d738,Uber,Black,17.0,1.00
693066,616d3611-1820-450a-9845-a9ff304a4842,Uber,UberXL,13.0,1.00
693067,633a3fc3-1f86-4b9e-9d48-2b7132112341,Uber,UberX,9.5,1.00
693069,727e5f07-a96b-4ad1-a2c7-9abc3ad55b4e,Uber,Black SUV,27.0,1.00


In [10]:
type(final_uber_lyft_df)

pandas.core.frame.DataFrame

In [11]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
final_uber_lyft_df.to_csv("uber_lyft_filtered.csv", encoding="utf-8", index=False)

In [12]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
#final_lyft_df.to_csv("lyft_filtered.csv", encoding="utf-8", index=False)

In [13]:
# Merge the datasets using the same columns
#uber_lyft_data = pd.merge(final_uber_df, final_lyft_df, how="left", left_on="id", right_on="State")
#uber_lyft_data = pd.merge(final_uber_df, final_lyft_df, how="left", left_on="id")

# Save the updated dataframe as a csv
#uber_lyft_data.to_csv("uber_lyft_merged_data.csv", encoding="utf-8", index=False)
#uber_lyft_data.head()

#booleans = []
#for Vehicle_Type in uber_lyft_df['name'].head():
#    if ((Vehicle_Type == "Lux" or Vehicle_Type == "Lyft XL")):
#        booleans.append(True)
#    else:
#        booleans.append(False)

In [14]:
#booleans[0:5]

In [15]:
#is_lux = pd.Series(booleans)
#is_lux.head()

In [16]:
#is_lux2 = final_uber_lyft_df.name == "Lux"
#is_lux2.head()

In [17]:
#is_lux2

In [18]:
#final_uber_lyft_df.loc

In [ ]:
x = final_uber_lyft_df['name']
y = final_uber_lyft_df['price']
#width = 0.25
#x_indexes = np.arange(len(x))
#plt.plot(x,y,label='First Line')
#plt.(x_indexes, y, color="k",width=width)
final_uber_lyft_df.plot(kind='bar', align='center', color='b')
plt.xlabel('Specific Cab Type/Name')
plt.ylabel('Price')
plt.title('Uber-Lyft data Lux-Black-XL only')
#plt.legend()
#plt.xticks(ticks=x_indexes, labels=x)
#plt.show()